# Publishing SQL Data as JSON Documents

## Publishing JSON
Up to this point, we have focused on JSON functions that allow you to extract values, objects, and arrays from documents. There are many circumstances where you want to be able to take the existing data in a table and make it available to outside world as JSON data. Publishing data as JSON is particularly useful when sending results back to an application that expects JSON.

Db2 provides two of the ISO SQL JSON publishing functions: `JSON_ARRAY` and `JSON_OBJECT`. The combination of these two functions can provide a way of generating most JSON documents.

### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

### Publishing Individual Values with JSON_OBJECT
The `JSON_OBJECT` function will generate a JSON object by creating `key:value` pairs. Using the ubiquitous `EMPLOYEE` table in the Db2 `SAMPLE` database, we can generate a JSON object that contains an employee’s first name through the following `JSON_OBJECT` function.

In [ ]:
%%sql -j
SELECT JSON_OBJECT(KEY 'lastname' VALUE lastname)
  FROM EMPLOYEE
FETCH FIRST ROW ONLY

The `JSON_OBJECT` function can have multiple levels of objects within it so you could create an object that contains an individual's entire name.

In [ ]:
%%sql -j
SELECT JSON_OBJECT(   
                   KEY 'identity' 
                   VALUE JSON_OBJECT(     
                                     KEY 'firstname' VALUE FIRSTNME,
                                     KEY 'lastname'  VALUE LASTNAME
                                     )
                   )
FROM EMPLOYEE
FETCH FIRST ROW ONLY;

You can generate extremely complex documents just using the `JSON_OBJECT` function. This function is complemented by the `JSON_ARRAY` function which allows grouping of values into an array. The syntax of the `JSON_OBJECT` function is shown below.

***JSON_OBJECT Syntax***
![JSON_VALUE](images/JSON_OBJECT.png)

***Key Expression***
![JSON_VALUE](images/jo-key-expression.png)

***Null Clause***
![JSON_VALUE](images/jo-null-clause.png)

***Unique Clause***
![JSON_VALUE](images/jo-unique-clause.png)

***Returning Clause***
![JSON_VALUE](images/jo-returning-clause.png)

What differentiates the `JSON_OBJECT` from other functions is that the `null-clause`, `unique-clause`, and `returning-clause` are for the entire block of `key-value` definitions not each individual one.

### Key Value Clause
The first clause is used to create the key-value pairs that you want published. You can create one or more key-value pairs, including the ability to nest objects. 

![JSON_OBJECT](images/jo-key-expression.png)

The `key-value` field represents the first field in a JSON object:
```json
"first_name": "Hronis"
```

The second field, *json-expression*, is the value associated with the key. The *json-expression* can be a reference to a column from a table, a constant, or a variable as long as it is not one of the following data types:
* GRAPHIC, VARGRAPHIC
* DBCLOB
* BINARY
* CHAR FOR BIT DATA, VARCHAR FOR BIT DATA
* XML

The simple example below will create a JSON document using some character constants.

In [ ]:
%sql -j VALUES JSON_OBJECT( KEY 'name' VALUE 'Bird');

You can use any of the valid Db2 data types in the `VALUE` clause as well as create multiple key-value pairs.

In [ ]:
%sql -j VALUES JSON_OBJECT( KEY 'name' VALUE 'Bird', KEY 'salary' VALUE 95000);

### Nested Key-value Expressions
The previous section illustrated the use of `JSON_OBJECT` to create a single-level document (no nesting). If you wanted to create a nested structure, then all you have to do is start another `JSON_OBJECT` function. The `VALUE` of the key is simply another `JSON_OBJECT` function call.

In [ ]:
%%sql -j
VALUES JSON_OBJECT( KEY 'foreword' VALUE
                 JSON_OBJECT( KEY 'primary' VALUE
                                  JSON_OBJECT( KEY 'first_name' VALUE 'Thomas',
                                               KEY 'last_name'  VALUE 'Hronis'
                                             )
                              FORMAT JSON
                            )
             FORMAT JSON
           );

There are no limits to the depth of nesting that you can code but it becomes difficult to keep track of levels as you build more complex `JSON_OBJECT` expressions!

### FORMAT JSON versus FORMAT BSON
Before discussing the use of the `FORMAT` clause, we need to understand the default output produced by the `JSON_OBJECT` function. The default output from the function is a character string that is surrounded by curly braces {} to form a proper JSON document.

In [ ]:
%sql -j VALUES JSON_OBJECT(KEY 'name' VALUE 'Bird');

When nesting a call to `JSON_OBJECT` within another one, by default, the upper `JSON_OBJECT` function places double quotes around the results from the inner function since it is returning a character string and these must be double-quoted by JSON format rules. For instance, the previous nested object example without any `FORMAT` clause produces the following output:

In [ ]:
%%sql -j
VALUES JSON_OBJECT( 
                   KEY 'author' VALUE
                   JSON_OBJECT( 
                               KEY 'first_name' VALUE 'Thomas',
                               KEY 'last_name'  VALUE 'Hronis'
                              )
                  );

This rather strange output is caused by two things that are occurring. First, the output from the second `JSON_OBJECT` function would be:

In [ ]:
%%sql -j 
VALUES JSON_OBJECT( 
                  KEY 'first_name' VALUE 'Thomas',
                  KEY 'last_name'  VALUE 'Hronis'
                 );

Since `FORMAT JSON` was not specified for the value in this `JSON_OBJECT` call, this output value is recognized only as a character string, not a JSON object, when it is passed as the proposed value for the key "author" in the first `JSON_OBJECT` function. As such, JSON format rules demand that the character value be enclosed in double quotes. And since this string itself contains double quote characters, the internal quote characters need to be escaped with the backslash character.
```json
"{\"first_name\":\"Thomas",\"last_name\":\"Hronis\"}"
```

Now this string is a proper JSON character value and can be used as the value in the first key-value pair to produce the final result:
```json
{"author":"{\"first_name\":\"Thomas\",\"last_name\":\"Hronis\"}"}
```
The `FORMAT` clause can be used to eliminate the external quotes and escape characters by indicating that the value to be returned by the nested `JSON_OBJECT` is actually already in valid JSON format and does not need to be treated as a character string. 

The following version of the example indicates that the value returned by the nested `JSON_OBJECT` call is in JSON format and would produce the desired output:

In [ ]:
%%sql -j 
VALUES JSON_OBJECT( 
                   KEY 'author' VALUE 
                       JSON_OBJECT(
                                   KEY 'first_name' VALUE 'Thomas',
                                   KEY 'last_name'  VALUE 'Hronis'
                                  ) 
                   FORMAT JSON
                  );

This `FORMAT` clause can be used when creating multi-level objects but not for simple values. If you use the clause with a single key-value pair, you will get an error.

In [ ]:
%sql -j VALUES JSON_OBJECT( KEY 'name' VALUE Bird FORMAT JSON)

This is because Bird by itself without the quotes is not in valid JSON format. As a JSON string data type, it should be enclosed in double quotes (e.g."Bird"). Recall our discussion on valid JSON format at the beginning of our journey!

The `FORMAT BSON` option will publish the key-value pair in binary format while `FORMAT JSON` will publish it character format. 

The following example illustrates the difference between using FORMAT JSON and no formatting directive.

In [ ]:
%%sql -j
SELECT JSON_OBJECT(
                   KEY 'identity' VALUE 
                       JSON_OBJECT( 
                                    KEY 'firstname' VALUE FIRSTNME,
                                    KEY 'lastname'  VALUE LASTNAME
                                   )
                   FORMAT JSON
                  )
FROM EMPLOYEE FETCH FIRST ROW ONLY;

By using the `FORMAT JSON` clause, we are able to eliminate the quoted strings from the output. If you are planning to publish the data back to an application that needs to process JSON, then you should always use `FORMAT JSON` (or `BSON`) to create the proper formatting for nested objects.

### NULL Handling
The `NULL` option on the `JSON_OBJECT` function is used to handle values that are null when retrieved from a table.

![JSON_OBJECT](images/jo-null-clause.png)

The default setting is `NULL ON NULL` which will publish the key-value pair even if the value is null.

In [ ]:
%%sql -j
VALUES JSON_OBJECT( 
                   KEY 'name' VALUE null, 
                   KEY 'salary' VALUE 95000
                   NULL ON NULL
                  );

In this case, when a null value was encountered in the result, the function put the JSON special word null in the output. Setting `ABSENT ON NULL` will prevent the key-value pair from being included in the output.

In [ ]:
%%sql -j
VALUES JSON_OBJECT( 
                   KEY 'name' VALUE null, 
                   KEY 'salary' VALUE 95000
                   ABSENT ON NULL
                  );

### KEYS
A best practice in generating key-value pairs is not to duplicate a key name at the same level. If there are duplicate keys within a document, there is no guarantee of which one will be chosen when you attempt to retrieve it.

![JSON_VALUE](images/jo-unique-clause.png)

The following `JSON_OBJECT` example creates two key-value pairs with the same key.

In [ ]:
%%sql -json
VALUES JSON_OBJECT( 
   KEY 'name' VALUE 'Thomas', 
   KEY 'name' VALUE 'Hronis' 
);

The default behavior is to ignore duplicate keys (`WITHOUT UNIQUE KEYS`) and so the above example will not generate an error. If you are positive that there will not be duplicate keys in your JSON, then you should leave this as the default since it will result in less overhead in the function.

When `WITH UNIQUE KEYS` is specified as part of the syntax, the function will raise an error code of -16407 for the above example.

In [ ]:
%%sql -json
VALUES JSON_OBJECT( 
   KEY 'name' VALUE 'Thomas', 
   KEY 'name' VALUE 'Hronis' 
   WITH UNIQUE KEYS
);

Note that duplicate keys can exist at different levels in an object and within arrays, as long as they have a unique JSON path expression.
```json
{
   "authors": [
               {"first_name": "Paul",  "last_name" : "Bird"},
               {"first_name": "George","last_name" : "Baklarz"}
              ],
   "foreword": {
               "primary": 
                         {
                           "first_name": "Thomas",
                           "last_name" : "Hronis"
                         }
               }
}
```

### RETURNING Clause
The `RETURNING` clause is used to define how the final JSON document is to be returned to the application. 
#### Returning Clause
![JSON_OBJECT](images/jo-returning-clause.png)

By default, the document is returned as a `CLOB` object, but you can use `CHAR`, `VARCHAR`, `CLOB`, `VARBINARY`, or `BLOB`. If you are returning the data as a character string, you must specify `FORMAT JSON` or use `FORMAT BSON` for a binary string.

If you supply too small of a data type, then the function will fail with an error message:
```sql
SQL0137N The length resulting from "SYSIBM.JSON_OBJECT" is greater than "20". SQLSTATE=54006 SQLCODE=-137
```

An error will also be produced if you try to return a `BSON` value into a character string:
```sql
SQL0171N The statement was not processed because the data type, length or value of the argument for the parameter in position "5" of routine "SYSIBM.JSON_OBJECT" is incorrect. Parameter name: "". SQLSTATE=42815 SQLCODE=-171
```

### Publishing Array Values with JSON_ARRAY
`JSON_OBJECT` is able to create complex JSON documents from data within a table, but it is not able to generate arrays. In order to create arrays, we must use the `JSON_ARRAY` function.

***JSON_ARRAY Syntax***
![JSON_OBJECT](images/JSON_ARRAY.png)

***JSON Expression***
![JSON_OBJECT](images/ja-json-expression.png)

***Full Select***
![JSON_OBJECT](images/ja-full-select.png)

***Null Clause***
![JSON_OBJECT](images/ja-null-clause.png)

***Returning Clause***
![JSON_OBJECT](images/ja-returning-clause.png)

There are two forms of the `JSON_ARRAY` function. The first version is similar to the `JSON_OBJECT` function where you supply a list of values to create an object. 

There is no key associated with a JSON array, so you only need to supply the list of values that you want in there. 

In [ ]:
%sql -j  VALUES JSON_ARRAY( 1523, 902, 'Thomas', 7777);

JSON array elements do not need to have the same data type – they can even contain other objects. Here is an example of a `JSON_OBJECT` being inserted into an array.

In [ ]:
%%sql -j
VALUES JSON_ARRAY(1523, 902, 
                  JSON_OBJECT( KEY 'lastname' VALUE 'Bird') FORMAT JSON, 
                  7777);

While the `JSON_ARRAY` function can be used by itself, it produces a JSON array value not a valid JSON object. The output from this function is meant to be used as part of a `JSON_OBJECT` structure.

The second form of the `JSON_ARRAY` function uses the results of a SQL select statement to build the array values. 

Only one `SELECT` statement can be used in the body of the function – you cannot have multiple `SELECT` commands in a list! If you do need to create an array from multiple sources, you should look at using a `SELECT` statement with `UNION` to create one list of items.

The following example publishes all of the department numbers for the departments that start with the letter B.

In [ ]:
%%sql -j
VALUES JSON_OBJECT(
                   KEY 'departments' VALUE 
                       JSON_ARRAY(SELECT DEPTNO FROM DEPARTMENT 
                                         WHERE DEPTNAME LIKE 'B%') 
                   FORMAT JSON
                  );

The `SELECT` statement can only return one column, otherwise an error message will be raised.
```sql
SQL0412N Multiple columns are returned from a subquery that is allowed only one column. SQLSTATE=42823 
SQLCODE=-412
```

If you do want to create an array of objects, you could use nested table expressions (or inline SQL) to generate the objects that you want. For instance, consider the `DEPARTMENT` table that we were using in the previous example. Perhaps you want to create an individual document to list all of the departments in the company, the document would look similar to the following.
```json
{
  "departments" : [
                    { 
                      "deptno" : "A01",
                      "deptname" : "Purchasing"
                    },
                    { 
                      "deptno" : "B01",
                      "deptname" : "Accounts"
                    }, 
                     …
                  ]
}
```
The `JSON_ARRAY` function can only work with one value, but what if we generate the object as part of another SQL statement? The `WITH` clause allows us to create the JSON document outside of the SQL that is publishing the data.

In [ ]:
%%sql -j 
WITH DEPARTMENTS(DEPT) AS 
(
 SELECT JSON_OBJECT(
                    KEY 'deptno' VALUE D.DEPTNO,
                    KEY 'deptname' VALUE D.DEPTNAME
                   )
 FROM DEPARTMENT D
 ORDER BY D.DEPTNO
)
SELECT * FROM DEPARTMENTS

Now we can select from the nested table expression as part of the `JSON_ARRAY` function to create an array of objects.

In [ ]:
%%sql -j
WITH DEPARTMENTS(DEPT) AS 
(
 SELECT JSON_OBJECT(
                    KEY 'deptno' VALUE D.DEPTNO,
                    KEY 'deptname' VALUE D.DEPTNAME
                   ) 
 FROM DEPARTMENT D 
 ORDER BY D.DEPTNO
)
SELECT JSON_OBJECT(      
                   KEY 'departments' VALUE 
                       JSON_ARRAY(
                                  SELECT DEPT FROM DEPARTMENTS 
                                  FORMAT JSON
                                 ) 
                   FORMAT JSON
                 )
FROM SYSIBM.SYSDUMMY1;

### FORMAT JSON versus FORMAT BSON
The `JSON_OBJECT` function had an extensive explanation of `FORMAT JSON` versus `FORMAT BSON`. If you are imbedding the `JSON_ARRAY` inside of a `JSON_OBJECT` function, then you should use `FORMAT JSON` to ensure the proper formatting of the document. The previous example which showed how to publish department numbers would have returned the following if `FORMAT JSON` was not used.

In [ ]:
%%sql -j
VALUES JSON_OBJECT(
                   KEY 'departments' VALUE 
                       JSON_ARRAY(SELECT DEPTNO FROM DEPARTMENT 
                                         WHERE DEPTNAME LIKE 'B%') 
                  );

### NULL Handling
The `NULL` option on the `JSON_ARRAY` function is used to handle arrays values that are null when retrieved from a table.

![JSON_OBJECT](images/ja-null-clause.png)
 
The default setting is `ABSENT ON NULL` which will ignore null values. Note that this is different from `JSON_OBJECT` which uses `NULL ON NULL` as the default.

In [ ]:
%sql -j VALUES JSON_ARRAY(1523, null);

If you really do want the `null` value, then you should include the `NULL ON NULL` option.

In [ ]:
%sql -j VALUES JSON_ARRAY(1523, null NULL ON NULL);

There is a potential that your SQL will not work if no values are returned by the `SELECT` statement. The `JSON_ARRAY` function expects at least one value to be returned (even a null value) in order to generate the array so you have to create a query to cover this possibility. The following SQL illustrates one technique that can be used.

In [ ]:
%%sql -j
VALUES JSON_OBJECT(
                   KEY 'departments' VALUE 
                       JSON_ARRAY(
                                   VALUES NULL
                                   UNION ALL
                                   SELECT DEPTNO FROM DEPARTMENT 
                                     WHERE DEPTNAME LIKE 'Z%'
                                 )
                   FORMAT JSON
                  );

The `VALUES NULL UNION ALL` will generate at least one value that is `null` in the list and then the `JSON_ARRAY` function can ignore it (remember that `ABSENT ON NULL` is the default behavior!) and generate an empty array.

### RETURNING Clause
The `RETURNING` clause is used to define how the final array is to be returned to the application. 
![JSON_OBJECT](images/ja-returning-clause.png)

By default, the document is returned as a `CLOB` object, but you can also use `CHAR`, `VARCHAR`, `CLOB`, `VARBINARY`, or `BLOB`. If you want the data to be in proper JSON format, then use `FORMAT JSON`. `FORMAT BSON` is not available as part of `JSON_ARRAY` since the expectation is that you will be using it within a `JSON_OBJECT` function and that handles the `FORMAT BSON` conversion.

If you supply too small of a data type, then the function will fail with an error message:
```sql
SQL0137N The length resulting from "SYSIBM.JSON_OBJECT" is greater than "20". SQLSTATE=54006 SQLCODE=-137
```

### Publishing Example
The first step is to decide what type of document you want to create. The `SAMPLE database` has the `EMPLOYEE` and `DEPARTMENT` table and we want to be able to publish a document that follows this format.
```json
{ 
  "empno" : "0001",
  "personal" : {
                  "first_name":"name","middle_initial":"x",
                  "last_name":"name","sex":"m","birthdate":"1999-01-01"
               },
  "compensation" : {"salary":50000,"bonus":4500,"commission":500},
  "position" : {"job":"worker","deptno":"A01","department":"cleaning"}.
  "manages" : ["A01"]}
}
```

The last field (manager) is a list of the departments that the individual manages. It should be empty if the employee is not a manager.

The following fields are available to help us build the JSON document.
* Employee number (EMPNO)
* Personal information (FIRSTNME, MIDINIT, LASTNAME, SEX, BIRTHDATE)
* Compensation (SALARY, BONUS, COMMISSION)
* Job details (JOB, WORKDEPT, DEPARTMENT NAME)
* Departments managed

All of the employee information comes from the `EMPLOYEE` table, while the department name is found in the DEPARTMENT table. The tricky portion is determining what departments report to a manager. 

There is a column in the `DEPARTMENT` table that gives us the manager number (`MGRNO`), their base department number (`DEPTNO`) and what the administrative department (or higher-level department) that manages them. 

The following `SELECT` statement will give us the manager numbers and the departments that report to them (including their own).

In [ ]:
%%sql 
SELECT D1.MGRNO, D1.DEPTNO FROM DEPARTMENT D1
  WHERE D1.MGRNO IS NOT NULL
UNION
SELECT D1.MGRNO, D2.DEPTNO FROM DEPARTMENT D1, DEPARTMENT D2
  WHERE D2.ADMRDEPT = D1.DEPTNO AND D1.MGRNO IS NOT NULL
ORDER BY MGRNO;

The JSON document we want to create can be broken down into five parts:
* Employee number
* Personal information
* Compensation
* Job Details
* Departments managed

The empno field is straightforward since it is a single value derived from the `EMPNO` field in the `EMPLOYEE`. 

In [ ]:
%%sql -j
SELECT JSON_OBJECT(
                   KEY 'empno' VALUE E.EMPNO 
                  )
FROM EMPLOYEE E FETCH FIRST ROW ONLY;

{
  "empno":"000010"  
}

The next three fields are JSON objects that we will need to create from a combination of the data in the `EMPLOYEE` and `DEPARTMENT` tables. The personal field is an object with five values. To create this object, we use the following JSON_OBJECT function.

In [ ]:
%%sql -j
SELECT JSON_OBJECT(
                   KEY 'first_name' VALUE E.FIRSTNME,
                   KEY 'middle_initial' VALUE E.MIDINIT,
                   KEY 'last_name' VALUE E.LASTNAME,
                   KEY 'sex' VALUE E.SEX,
                   KEY 'birthdate' VALUE E.BIRTHDATE
                   NULL ON NULL
                  )
FROM EMPLOYEE E FETCH FIRST ROW ONLY;

This object needs to be nested into the SQL statement we are building for our desired JSON document. The next example shows the two SQL statements merged together.

In [ ]:
%%sql -j
SELECT JSON_OBJECT(
                   KEY 'empno' VALUE E.EMPNO, 
                   KEY 'personal' VALUE
                       JSON_OBJECT(
                                   KEY 'first_name' VALUE E.FIRSTNME,
                                   KEY 'middle_initial' VALUE E.MIDINIT,
                                   KEY 'last_name' VALUE E.LASTNAME,
                                   KEY 'sex' VALUE E.SEX,
                                   KEY 'birthdate' VALUE E.BIRTHDATE
                                  ) 
                   FORMAT JSON
                  )
FROM EMPLOYEE E FETCH FIRST ROW ONLY;

We have to remember to add the `FORMAT JSON` clause to remove the escape characters for any double quotes in the result from the upper `JSON_OBJECT` function.

The same process has to be used to create the `JSON_OBJECT` functions for the two other sections (compensation and position). To generate the department name, we must add a join in the SQL between the `EMPLOYEE` table and the `DEPARTMENT` table in the `WHERE` clause. 

The SQL to generate everything except the manager list is found below: 

In [ ]:
%%sql -j
SELECT JSON_OBJECT(
                   KEY 'empno' VALUE E.EMPNO, 
                   KEY 'personal' VALUE 
                       JSON_OBJECT(
                                   KEY 'first_name' VALUE E.FIRSTNME,
                                   KEY 'middle_initial' VALUE E.MIDINIT,
                                   KEY 'last_name' VALUE E.LASTNAME,
                                   KEY 'sex' VALUE E.SEX,
                                   KEY 'birthdate' VALUE E.BIRTHDATE
                                  )
                       FORMAT JSON,
                   KEY 'compensation' VALUE 
                       JSON_OBJECT(
                                   KEY 'salary' VALUE E.SALARY,
                                   KEY 'bonus' VALUE E.BONUS,
                                   KEY 'commission' VALUE E.COMM
                                  ) 
                       FORMAT JSON,  
                   KEY 'position' VALUE 
                       JSON_OBJECT(
                                   KEY 'job' VALUE E.JOB,
                                   KEY 'deptno' VALUE E.WORKDEPT,
                                   KEY 'department' VALUE D.DEPTNAME   
                                  ) 
                       FORMAT JSON
                  )
FROM EMPLOYEE E, DEPARTMENT D
WHERE D.DEPTNO = E.WORKDEPT FETCH FIRST ROW ONLY;

Finally, we need to add the list of departments that a manager is responsible for. The code for determining this was shown earlier. In order to get this into the JSON document, we must use the `JSON_ARRAY` function with the imbedded SQL statement. This SQL snippet shows the results from manager `E.EMPNO = '000010'`.

In [ ]:
%%sql -j
SELECT JSON_ARRAY(
           VALUES NULL
            UNION 
           SELECT D1.DEPTNO FROM DEPARTMENT D1 WHERE D1.MGRNO = E.EMPNO
            UNION 
           SELECT D2.DEPTNO FROM DEPARTMENT D1, DEPARTMENT D2 
             WHERE D2.ADMRDEPT = D1.DEPTNO AND D1.MGRNO = E.EMPNO 
          ) FROM EMPLOYEE E
WHERE E.EMPNO = '000010'

We need to add the `VALUES NULL` clause to make sure we have a null array created in the event the employee is not a manager. The final SQL is found below.

In [ ]:
%%sql -j
SELECT JSON_OBJECT(
                   KEY 'empno' VALUE E.EMPNO, 
                   KEY 'personal' VALUE 
                       JSON_OBJECT(
                                   KEY 'first_name' VALUE E.FIRSTNME,
                                   KEY 'middle_initial' VALUE E.MIDINIT,
                                   KEY 'last_name' VALUE E.LASTNAME,
                                   KEY 'sex' VALUE E.SEX,
                                   KEY 'birthdate' VALUE E.BIRTHDATE
                                  ) 
                        FORMAT JSON,
                   KEY 'compensation' VALUE 
                       JSON_OBJECT(
                                   KEY 'salary' VALUE E.SALARY,
                                   KEY 'bonus' VALUE E.BONUS,
                                   KEY 'commission' VALUE E.COMM
                                  ) 
                       FORMAT JSON,  
                   KEY 'position' VALUE 
                       JSON_OBJECT(
                                   KEY 'job' VALUE E.JOB,
                                   KEY 'deptno' VALUE E.WORKDEPT,
                                   KEY 'department' VALUE D.DEPTNAME   
                                  ) 
                       FORMAT JSON,
                   KEY 'manages' VALUE 
                       JSON_ARRAY(
                                  VALUES NULL
                                  UNION 
                                  SELECT D1.DEPTNO FROM DEPARTMENT D1
                                    WHERE D1.MGRNO = E.EMPNO 
                                  UNION 
                                  SELECT D2.DEPTNO FROM DEPARTMENT D1,
                                         DEPARTMENT D2
                                    WHERE D2.ADMRDEPT = D1.DEPTNO AND
                                          D1.MGRNO = E.EMPNO       
                                 ) 
                       FORMAT JSON
                  )
FROM EMPLOYEE E, DEPARTMENT D
WHERE D.DEPTNO = E.WORKDEPT;

## Summary
The `JSON_OBJECT` and `JSON_ARRAY` functions can be used to create JSON documents from data in relational tables. The `JSON_OBJEC`T function can be used to create complex objects, by nesting other `JSON_OBJECT` and `JSON_ARRAY` calls. 

To create a JSON document, you need to decide what the structure of your document will look like, the fields that you want to publish, and also decide what other tables you may need to create arrays. 

The best way to publish relational data as JSON is to break up the document into multiple sections to create and test them individually before combining them all together to get the desired end result.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]